## Problem Set 2: Merging and regular expressions

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset2_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing


In [2]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset2_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [3]:
debar_df = pd.read_csv("pset2_inputdata/debar.csv")
debar_df.head()


,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [4]:
# number rows in debar 
len(debar_df)

# number of unique employer names 
debar_df["Name"].nunique()

114

98

There are multiple rows for some employers.

## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [5]:
name_counts = debar_df["Name"].value_counts()
repeated_names = name_counts[name_counts > 1].index
debar_df["is_repeated"] = np.where(debar_df["Name"].isin(repeated_names), True, False) 


In [6]:
debar_df[debar_df["is_repeated"] == True]

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
17,"Maple Ridge Custom Services, LLC","Altheimer, AK",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,True
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True


In [7]:
mult_debar = debar_df[debar_df["is_repeated"] == True]
mult_debar.head()
mult_debar.shape
mult_debar

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True


(32, 7)

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
17,"Maple Ridge Custom Services, LLC","Altheimer, AK",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,True
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True


## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [8]:
# cleaning state names 
state_replacements = {
    "Utah": "UT",
    "Texas": "TX",
    "AK": "AR",
    "Georgia": "GA"
}

# function to replace names with abreviations 
def replace_state(match):
    state_name = match.group(1)
    return state_replacements.get(state_name, "")

# Cleaning state name pattern
state_pattern = r"(Utah|Texas|AK|Georgia)"

# clean names 
mult_debar.loc[:, "City, State"] = mult_debar["City, State"].apply(lambda x: re.sub(state_pattern, replace_state, x))

# switch brownsville to brownsfield 
mult_debar["City, State"] = mult_debar["City, State"].str.replace("Brownfield, TX", "Brownsville, TX")



# make pivot table for duplicates
mult_debar_wide = pd.pivot_table(mult_debar,
                                 values='Start date',
                                 index=["Name", "City, State"],
                                 columns=mult_debar.groupby(["Name", "City, State"]).cumcount() + 1,
                                 aggfunc='first')
mult_debar_wide.reset_index(inplace=True)

mult_debar_wide.columns = ["Name", "City, State", "start_date_viol1", "start_date_viol2"]



/var/folders/sp/m42g974s16vdc06dp0p_mzmw0000gn/T/ipykernel_6285/3677858580.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_debar["City, State"] = mult_debar["City, State"].str.replace("Brownfield, TX", "Brownsville, TX")


In [12]:
mult_debar_wide.head()
mult_debar_wide.shape 
mult_debar_wide

,Name,"City, State",start_date_viol1,start_date_viol2,is_dup
0,Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014,True
1,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014,True
2,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014,True
3,Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015,False
4,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018,True


(16, 5)

,Name,"City, State",start_date_viol1,start_date_viol2,is_dup
0,Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014,True
1,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014,True
2,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014,True
3,Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015,False
4,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018,True
5,F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014,True
6,Loewen Harvesting LLC,"Brownsville, TX",8/20/2014,8/20/2014,True
7,Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015,True
8,"Maple Ridge Custom Services, LLC","Altheimer, AR",11/16/2014,11/16/2014,True
9,Mark Duncan,"Roosevelt, UT",11/16/2014,11/16/2014,True


## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [14]:
mult_debar_wide["is_dup"] = np.where(mult_debar_wide.start_date_viol1 == mult_debar_wide.start_date_viol2, True, False)
mult_debar_wide

duplicates = mult_debar_wide[mult_debar_wide['is_dup'] == True]["Name"]

subset_dup = debar_df[debar_df.Name.isin(duplicates)]
subset_not_dup = debar_df[~debar_df.Name.isin(duplicates)]


subset_dup.shape
subset_dup.drop_duplicates("Name", inplace=True)
subset_dup.shape




debar_clean = pd.concat([subset_dup, subset_not_dup], axis=0)

debar_clean

,Name,"City, State",start_date_viol1,start_date_viol2,is_dup
0,Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014,True
1,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014,True
2,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014,True
3,Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015,False
4,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018,True
5,F&W Farms,"Ingalls, KS",12/10/2014,12/10/2014,True
6,Loewen Harvesting LLC,"Brownsville, TX",8/20/2014,8/20/2014,True
7,Macky and Brad Farms,"Plains, TX",2/13/2015,2/13/2015,True
8,"Maple Ridge Custom Services, LLC","Altheimer, AR",11/16/2014,11/16/2014,True
9,Mark Duncan,"Roosevelt, UT",11/16/2014,11/16/2014,True


(26, 7)

/var/folders/sp/m42g974s16vdc06dp0p_mzmw0000gn/T/ipykernel_6285/3903265484.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_dup.drop_duplicates("Name", inplace=True)


(13, 7)

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True
...,...,...,...,...,...,...,...
107,Walker Place,"Danville, IL",Failure to comply with the employer's obligati...,2 months,11/19/2019,1/26/2020,False
108,County Fair Farm (company) and Andrew Williams...,"Jefferson, ME",WHD Debarment,3 years,3/8/2017,3/8/2020,False
110,Jesus Ledesma,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,False
112,B & R Harvesting and Paul Cruz (individual),"Collins, Georgia",WHD Debarment,3 years,4/9/17,4/9/20,False


# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset4_inputdata`

In [15]:
jobs_df = pd.read_csv("pset2_inputdata/jobs.csv")


##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [16]:
exact_matches = pd.merge(jobs_df, debar_clean, left_on="EMPLOYER_NAME", right_on="Name", how="inner")
exact_matches

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,Rafael Barajas,NaN,...,NaN,https://seasonaljobs.dol.gov/,7,Rafael Barajas,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,False


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [17]:
## insert your code to turn into uppercase here
jobs_df["EMPLOYER_NAME"] = [name.upper() for name in jobs_df["EMPLOYER_NAME"]]
debar_clean["Name"] = [name.upper() for name in debar_clean["Name"]]

In [18]:
## insert your code for the random sample
jobs_df.sample(n=15)
debar_clean.sample(n=15)

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
2112,H-300-20316-908341,Determination Issued - Certification,2020-11-19 15:31:41.230,2020-12-22 00:00:00.000,Individual Employer,N,Seasonal,N,CHRIS LEMIONE JR FARMS,NaN,...,N,1,N,NaN,12.68,55.0,13183592224,clemoinejr@gmail.com,www.louisianaworks.net,5
1820,H-300-20339-938534,Determination Issued - Certification,2020-12-04 18:39:28.160,2020-12-15 00:00:00.000,Association - Agent,N,Seasonal,N,WARREN W. ROBERTS,NaN,...,Y,2,N,NaN,12.68,55.0,13074722105,info@mpaswy.com,NaN,0
422,H-300-20260-828070,Determination Issued - Denied,2020-09-22 13:56:41.030,2020-10-21 00:00:00.000,Individual Employer,Y,Seasonal,N,"RB HARVESTING, LLC","RB Harvesting, LLC",...,N,1,N,NaN,12.68,55.0,12294540518,rbharvestingh2a@gmail.com,www.employga.com,1
1044,H-300-20309-897617,Determination Issued - Certification,2020-11-05 17:34:53.130,2020-11-20 00:00:00.000,Individual Employer,N,Seasonal,N,"JUBILEE FARMS, INC","Jubilee Farms, Inc",...,Y,2,Y,4.22,12.68,55.0,16622071150,lhjubileefarmsinc@live.com,NaN,1
1041,H-300-20307-894204,Determination Issued - Certification,2020-11-02 20:42:18.200,2020-11-20 00:00:00.000,Individual Employer,N,Seasonal,N,LOV RANCH,NaN,...,N,1,N,NaN,12.68,55.0,19708784686,bbrennan@wreawildblue.org,www.connectingcolorado.com,0
1883,H-300-20337-933812,Determination Issued - Certification,2020-12-01 22:56:16.650,2020-12-16 00:00:00.000,Association - Agent,N,Seasonal,N,ROBERT J HARLAN,NaN,...,N,1,N,NaN,12.68,55.0,13074722105,info@mpaswy.com,NaN,0
2394,H-300-20307-894806,Determination Issued - Certification,2020-11-05 10:26:46.530,2020-12-29 00:00:00.000,Individual Employer,N,Seasonal,N,AYDANI WHOLESALE NURSERY LLC,NaN,...,N,1,N,NaN,12.68,55.0,NaN,farmjobs@nach2a.com,https://seasonaljobs.dol.gov/,0
34,H-300-20258-821509,Determination Issued - Withdrawn,2020-09-18 13:37:01.130,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,"ELW FARMS, INC.",NaN,...,N,1,N,NaN,12.68,55.0,16208741160,gwenhpec@hotmail.com,NaN,1
1557,H-300-20295-884349,Determination Issued - Certification,2020-11-17 13:15:02.670,2020-12-09 00:00:00.000,Individual Employer,N,Seasonal,N,XCEL FARMS LLC,NaN,...,Y,2,N,NaN,12.68,55.0,17856141489,cmorehead@xcelgrp.com,NaN,0
1676,H-300-20317-910686,Determination Issued - Certification,2020-11-19 11:17:16.490,2020-12-11 00:00:00.000,Individual Employer,N,Seasonal,N,DESERT CREEK HONEY LLC,NaN,...,N,1,N,NaN,12.68,55.0,12145783327,blake@desertcreekhoney.com,NaN,0


,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
42,LEONARD SMITH FARMS,"DeRidder, LA",Impeding the Audit Process – Non- Response,2 years,6/19/2015,6/18/2017,False
89,XAVIER HORNE,"Lyons, Georgia",Failure to respond to audit request,2 years,9/27/2017,9/26/2019,True
100,EVERGREEN PRODUCE,"Adel, GA",Failure to comply with the employer's obligati...,2 years,10/29/2017,10/29/2019,False
15,SRT FARMS,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
18,F&W FARMS,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True
43,"HAITI/USA WORKFORCE, LLC","Miami, FL",Impeding the Audit Process – Non- Response,2 years,6/19/2015,6/18/2017,False
22,ALTERIC JEAN-CHARLES,"Fort Lauderdale, FL",Non-payment,1 year,12/6/2014,12/5/2015,False
31,MACKY AND BRAD FARMS,"Plains, TX",Failure to respond to audit (no response),1 year,2/13/2015,2/12/2016,True
52,LOUIE M. ASUMENDI,"Nampa, ID",Impeding the Audit Process – Non- Response,2 years,3/11/2015,3/10/2017,False
67,ROSALVA GARCIA,"Dade City, Florida",Failure to respond to audit request,2 years,7/27/2016,6/27/2018,False


In [ ]:
## insert your code for assigning the uppercase names back to the data

### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [19]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [27]:
## insert your code here with the regex pattern for part A
pattern = r"(INC|LLC|CO)\."

## insert your code to use re.sub to apply the pattern to the test cases for part B
re.sub(pattern, r"\1", pos_example_1)
re.sub(pattern, r"\1", pos_example_2)
re.sub(pattern, r"\1", neg_example)

'CISCO PRODUCE INC'

'AVOYELLES HONEY CO, LLC'

'E.V. RANCH LLP'

### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [28]:
## your code here to clean the columns
debar_clean["name_clean"] = debar_clean["Name"].apply(lambda name: re.sub(pattern, r"\1", name))
jobs_df["name_clean"] = jobs_df["EMPLOYER_NAME"].apply(lambda name: re.sub(pattern, r"\1", name))

changed_names_debar = debar_clean[debar_clean['Name'] != debar_clean['name_clean']]
changed_names_jobs = jobs_df[jobs_df['EMPLOYER_NAME'] != jobs_df['name_clean']]


In [29]:
## your code here to print the head
changed_names_debar[["Name", "name_clean"]].head()
changed_names_jobs[["EMPLOYER_NAME", "name_clean"]].head()

,Name,name_clean
3,ANTON FERTILIZER INC.,ANTON FERTILIZER INC
4,"GREAT PLAINS FLUID SERVICE, INC.","GREAT PLAINS FLUID SERVICE, INC"
5,PROMAX INC.,PROMAX INC
13,REIMER'S INC.,REIMER'S INC
19,CISCO PRODUCE INC.,CISCO PRODUCE INC


,EMPLOYER_NAME,name_clean
4,"DUNSON HARVESTING, INC.","DUNSON HARVESTING, INC"
7,"FARM LABOR ASSOCIATION FOR GROWERS, INC.","FARM LABOR ASSOCIATION FOR GROWERS, INC"
14,"MCLAIN FARMS, INC.","MCLAIN FARMS, INC"
17,"BONNIE PLANTS, INC.","BONNIE PLANTS, INC"
18,"B & W QUALITY GROWERS, INC.","B & W QUALITY GROWERS, INC"


## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate why talked about fuzzy matching. Fuzzy matching could recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical.

In [30]:
## your code here

jobs_debar = pd.merge(jobs_df, debar_clean, on="name_clean", how="inner")
jobs_debar



,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,name_clean,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,https://seasonaljobs.dol.gov/,7,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,False


In [31]:
regex_pattern = r"(\.)|(\sLLP)"

re.sub(regex_pattern, r"", "SLASH E.V. RANCH LLP") 

debar_clean["name_clean_2"] = debar_clean["name_clean"].apply(lambda name: re.sub(regex_pattern, r"\1", name))


final_joined = pd.merge(jobs_df, debar_clean, left_on="name_clean", right_on="name_clean_2", how="left", indicator=True)
print(final_joined[final_joined["_merge"]== "both"].shape[0])

'SLASH EV RANCH'

1


# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [43]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern
extra_credit_regex = r"((*\w*\s+(COMPANY))|((*\w*s+)INDIVIDUAL)"
## your code here to apply it to the pos_example
re.findall(extra_credit_regex, pos_example)
## your code here to apply it to the negative example

error: nothing to repeat at position 2

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [ ]:
# your code here

   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [ ]:
# your code here

# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data

In [ ]:
## your code here